In [1]:
print('Takumi House & Kailen Santos')

Takumi House & Kailen Santos


In [2]:
from matplotlib import pyplot as plt
import matplotlib as mp
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import sklearn.manifold
from scipy import cluster
import seaborn as sns
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
import requests
import json
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
import re
import json
import pickle
%matplotlib inline

In [3]:
##load in pickled dataframe of all the reviews

#try:
#    with open('df_reviews','rb') as f:
#        df_reviews = pickle.load(f)
        
#except FileNotFoundError:
#    with open('df_reviews','wb') as f:
#        pickle.dump(df_reviews, f)

In [4]:
##load in pickled stemmed data for the review text
try:
    with open('stemmed_data','rb') as f:
        stemmed_data = pickle.load(f)
        
except FileNotFoundError:
    stemmed_data = [" ".join(SnowballStemmer("english", ignore_stopwords=True).stem(word)  
        for sent in sent_tokenize(message)
        for word in word_tokenize(sent))
        for message in df_reviews['text']]
    
    with open('stemmed_data','wb') as f:
        pickle.dump(stemmed_data, f)

In [5]:
df = pd.read_csv('Restaurant_Inspections.csv',header=0)
# df = df[df.City == 'Las Vegas']
# df = (df[df['Inspection Result'] == 'Closed with Fees']).append(df[df['Inspection Result'] == 'Closed without Fees'])

df.shape

#with open ('Resaurant_Inspections.csv') as file:
#   for line in file:
#      if (line[6]=='Las Vegas' && line[]

/Users/takumihouse/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(123919, 23)

In [6]:
df = df[df.City == 'Las Vegas']
df.shape

(94700, 23)

In [7]:
categories = df['Category Name'].unique().tolist()
print(categories)

['Pantry', 'Special Kitchen', 'Garde Manger', 'Kitchen Bakery', 'Bar / Tavern', 'Buffet', 'Snack Bar', 'Restaurant', 'Banquet Support', 'Portable Unit', 'Banquet Kitchen', 'Portable Bar', 'Meat/Poultry/Seafood', 'Vegetable Prep', 'Caterer', 'Barbeque', 'Confection', 'Concessions', 'Produce Market', 'Institutional Food Service', 'Elementary School Kitchen', 'Self-Service Food Truck', 'Bakery Sales', 'Food Trucks / Mobile Vendor', 'Grocery Store Sampling', 'Beer Bar', 'Childcare Kitchens', "Farmer's Market", 'Main Kitchen', 'Frozen Meat Sales']


In [8]:
categories.remove('Elementary School Kitchen')
categories.remove('Grocery Store Sampling')
categories.remove('Farmer\'s Market')
categories.remove('Caterer')
categories.remove('Concessions')
categories.remove('Institutional Food Service')
categories.remove('Banquet Support')
categories.remove('Snack Bar')

In [9]:
df = df[df['Category Name'].isin(categories)]
df.shape

(82656, 23)

In [10]:
coord_strings = df['Location 1'].tolist()

In [11]:
import ast
xcoords = []
ycoords = []
for coord in coord_strings:
    xcoords.append(ast.literal_eval(coord)[0])
    ycoords.append(ast.literal_eval(coord)[1])

In [12]:
count = 0
for idx, ycoord in enumerate(ycoords):
    if ycoord > 0:
        ycoords[idx] = ycoord * -1
        count += 1
print('{} coordinates fixed.'.format(count))

510 coordinates fixed.


In [13]:
df['Coordinates'] = list(zip(xcoords, ycoords))
df.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Employee ID,Inspection Type,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates
11,DA0001715,PR0000426,CAESARS AUGUSTUS BUTLER PANTRY,CAESARS PALACE HOTEL & CASINO,Pantry,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000417,Routine Inspection,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
12,DA0001716,PR0000426,CAESARS AUGUSTUS BUTLER PANTRY,CAESARS PALACE HOTEL & CASINO,Pantry,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000567,Routine Inspection,2,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
13,DA0001728,PR0000427,CAESARS DIAMOND LOUNGE KITCHEN,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,8,...,EE7000417,Routine Inspection,4,A,NaN,Compliant,14,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
14,DA0001773,PR0000434,CAESARS BAKERY WAREWASH,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000417,Routine Inspection,7,A,NaN,Compliant,14232831,01/22/2014 02:03:44 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
15,DA0001776,PR0000434,CAESARS BAKERY WAREWASH,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000392,Routine Inspection,2,A,NaN,Compliant,NaN,01/22/2014 02:03:44 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"


In [14]:
coordinates_list = df['Coordinates'].tolist()
xcoords = []
ycoords = []
for coordinates in coordinates_list:
    xcoords.append(coordinates[0])
    ycoords.append(coordinates[1])
df['X'] = xcoords
df['Y'] = ycoords

In [15]:
df = df[df['Y'] < -114.5]
df = df[df['Y'] > -115.6]
df_clean = df.reset_index(drop = True)

In [16]:
coordinates_list = df_clean['Coordinates'].tolist()
xcoords = []
ycoords = []
for coordinates in coordinates_list:
    xcoords.append(coordinates[0])
    ycoords.append(coordinates[1])

In [17]:
df = df_clean[df_clean['Date Current']==df_clean['Inspection Date']]
# df['XY'] = list(zip(df.X, df.Y))
df.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates,X,Y
60,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,4,...,4,A,NaN,Compliant,114,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)",36.116156,-115.175058
187,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,10,...,10,A,NaN,Compliant,64114,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
243,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,3,...,3,A,NaN,Compliant,13,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
306,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,Las Vegas,Nevada,89101-5601,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1713005, -115.1462312)","(36.1713005, -115.1462312)",36.171301,-115.146231
410,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,Las Vegas,Nevada,89101,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1800444, -115.131106)","(36.1800444, -115.131106)",36.180044,-115.131106


In [18]:
# df_sorted = df.sort(columns = 'XY', axis = 0)
# df_sorted.head(n=40)

In [19]:
df_clean = df.reset_index(drop=True)
df_clean.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates,X,Y
0,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,4,...,4,A,NaN,Compliant,114,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)",36.116156,-115.175058
1,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,10,...,10,A,NaN,Compliant,64114,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
2,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,3,...,3,A,NaN,Compliant,13,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
3,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,Las Vegas,Nevada,89101-5601,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1713005, -115.1462312)","(36.1713005, -115.1462312)",36.171301,-115.146231
4,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,Las Vegas,Nevada,89101,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1800444, -115.131106)","(36.1800444, -115.131106)",36.180044,-115.131106


In [20]:
# df_agg = df_clean.groupby('Restaurant Name',as_index=False).agg(lambda x: x.tolist())
df_agg = df_clean.copy()

In [21]:
df_agg.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates,X,Y
0,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,4,...,4,A,NaN,Compliant,114,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)",36.116156,-115.175058
1,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,10,...,10,A,NaN,Compliant,64114,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
2,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,3,...,3,A,NaN,Compliant,13,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
3,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,Las Vegas,Nevada,89101-5601,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1713005, -115.1462312)","(36.1713005, -115.1462312)",36.171301,-115.146231
4,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,Las Vegas,Nevada,89101,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1800444, -115.131106)","(36.1800444, -115.131106)",36.180044,-115.131106


In [22]:
dfcategories = df_agg.columns
dfcategories

Index(['Serial Number', 'Permit Number', 'Restaurant Name', 'Location Name',
       'Category Name', 'Address', 'City', 'State', 'Zip', 'Current Demerits',
       'Current Grade', 'Date Current', 'Inspection Date', 'Inspection Time',
       'Employee ID', 'Inspection Type', 'Inspection Demerits',
       'Inspection Grade', 'Permit Status', 'Inspection Result', 'Violations',
       'Record Updated', 'Location 1', 'Coordinates', 'X', 'Y'],
      dtype='object')

In [23]:
# drop unnecessary columns for categorization step later
df_agg = df_agg.drop(['Serial Number'], axis = 1)
df_agg = df_agg.drop(['City'], axis = 1)
df_agg = df_agg.drop(['State'], axis = 1)
df_agg = df_agg.drop(['Zip'], axis = 1)
df_agg = df_agg.drop(['Inspection Time'], axis = 1)
df_agg = df_agg.drop(['Employee ID'], axis = 1)
df_agg = df_agg.drop(['Inspection Type'], axis = 1)
df_agg = df_agg.drop(['Location 1'], axis = 1)
df_agg = df_agg.drop(['Coordinates'], axis = 1)
df_agg = df_agg.drop(['Record Updated'], axis = 1)
df_agg = df_agg.drop(['Inspection Date'], axis = 1)

In [24]:
df_agg.head()

,Permit Number,Restaurant Name,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,4,A,05/17/2010 12:00:00 AM,4,A,NaN,Compliant,114,36.116156,-115.175058
1,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,10,A,07/30/2010 12:00:00 AM,10,A,NaN,Compliant,64114,36.120601,-115.176838
2,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,3,A,07/30/2010 12:00:00 AM,3,A,NaN,Compliant,13,36.120601,-115.176838
3,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,0,A,03/16/2010 12:00:00 AM,0,A,NaN,Compliant,NaN,36.171301,-115.146231
4,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,0,A,05/04/2011 12:00:00 AM,0,A,NaN,Compliant,NaN,36.180044,-115.131106


In [25]:
df_agg.shape

(12926, 15)

In [26]:
# dfcategories = df_agg.columns
# for x in dfcategories[1:]:
#     for y in range(len(df_agg[x])):
#         df_agg[x][y] = list(set(df_agg[x][y]))
#         df_agg[x][y] = df_agg[x][y][0]

In [27]:
df_agg.head()

,Permit Number,Restaurant Name,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,4,A,05/17/2010 12:00:00 AM,4,A,NaN,Compliant,114,36.116156,-115.175058
1,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,10,A,07/30/2010 12:00:00 AM,10,A,NaN,Compliant,64114,36.120601,-115.176838
2,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,3,A,07/30/2010 12:00:00 AM,3,A,NaN,Compliant,13,36.120601,-115.176838
3,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,0,A,03/16/2010 12:00:00 AM,0,A,NaN,Compliant,NaN,36.171301,-115.146231
4,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,0,A,05/04/2011 12:00:00 AM,0,A,NaN,Compliant,NaN,36.180044,-115.131106


In [28]:
try:
    with open('df_agg_revs','rb') as f:
        df_agg_revs = pickle.load(f)
        
except FileNotFoundError:
    with open('df_agg_revs','wb') as f:
        pickle.dump(df_agg_revs, f)

In [29]:
sick_ids = []
for x in df_agg_revs['business_id'].items():
    sick_ids.append(x[1])
print(sick_ids)

['F8ezW6hYgmQYBmXOVjhytQ', 'atLmZg76uDg0oKhKjHLMyg', 'y7mody_dVe_2K8-wnnPtUw', 'Su2-lf8TbowpHZcS3XIRfQ', 'xoYY0F5hrs00h285tWmZZA', 'KuXGis6uUuhpSKmD1s5Zvw', 'y_L32uJc_sPupdjhb2YO-g', 'stZBSJLsrwrikYUkZ4aAAA', 'lSS9k3Fbs8E8qEVTy4-d5w', 'RnuCCKK-5Ezfz0xv0F2kag', 'dVWP9QbfrKkMgSySiaY4HA', 'hsgTsHqun5t2vZyGNMILZg', '91ohFfAbLvc7HcI68OHy3w', '-CIZh4JOZuOEbSzLVrAIGw', 'i7HvZ1yTTIaR6bfa_3jGEw', 'SsTxjxo8qvqBMvan1rzNzg', '7fuiTVuFt7CTAQRpFQf8DA', 'fLQ9XFlHtYHdAKCS4cSMag', 'XbnnsZKL8AohGRzo1dCFCQ', 'mdOsYpHnL_faguImkNbheA', 'ivi_LQZswT5xBpqXsuqK1Q', 'gcY4e5P1aac-Cj0NePNmPQ', 'XDk9FglRpw4TzhWejKwP4g', 'wLmMGlGCRdNSxArKca6DJA', 'aTwYIgWQUfoa2HobcjodTw', 'AFDFOs2GaIyUArGyEUDQzQ', 'botAdj1CUl2wA_Dz1Y-kIw', 'c8b7niizd3jigDlq-z_E6Q', 'VRpis8ZOVDFyO0WZAW5Rog', 'ylYhDUedhhG4WlcjvFq4Cw', 'PBkdlqw53cc2AZsfFD8AEQ', '94HshaJUV3DLvXkvbCbZqg', 'tFU2Js_nbIZOrnKfYJYBBg', 'KAxevTSEIuw99MUKnlzw5Q', 'RsI0wYUbWIHrHtOGZ54stg', 'xZoyDn7ekD4ue_liS_nqMA', 'RUz4d1D83xRRXcc-ffkM5g', 'ySpNcdz0tSpaPNqefAbfOw', 'pcIPHOoQZu

In [30]:
file = 'yelp_academic_dataset_business.json'
business_locations = {}
addresses = []
names=[]
with open(file) as f:
    for line in f:
        r_line = json.loads(line)
        if r_line['business_id'] in set(sick_ids):
            business_locations.update({r_line['business_id']:(r_line['full_address']).rsplit('\n', 5)[0].lower()})
            addresses.append(r_line['full_address'].rsplit('\n', 5)[0].lower())
            names.append(r_line['name'].lower())
addresses = np.asarray(addresses)
df_agg_revs['address']=addresses
df_agg_revs['name']=names
#print(business_locations)

In [31]:
df_agg_revs.head()

,business_id,stars,text,votes,name,address
address,,,,,,
1916 village center cir,F8ezW6hYgmQYBmXOVjhytQ,"[3, 4, 2, 4, 3, 4, 1, 1, 1, 3, 4, 5, 1, 3, 4]",[This Jack's is not on the glitzy Las Vegas St...,"[{'useful': 4, 'funny': 3, 'cool': 4}, {'usefu...",chianti cafe,1916 village center cir
2737 las vegas blvd n,atLmZg76uDg0oKhKjHLMyg,"[2, 5, 1, 1, 5, 4, 3, 1, 1, 2, 1, 5, 3, 2, 2, ...",[Open 24 hours a day. Carna asada burrito is p...,"[{'useful': 1, 'funny': 0, 'cool': 0}, {'usefu...",rigos taco,2737 las vegas blvd n
2312 e craig rd,y7mody_dVe_2K8-wnnPtUw,"[5, 4, 4, 4, 5, 5, 5, 2, 4, 5, 5, 4, 5, 4, 4, ...","[The first time I was here, I ordered the Hawa...","[{'useful': 2, 'funny': 0, 'cool': 2}, {'usefu...",mulligans bar & grill,2312 e craig rd
3100 e lake mead blvd,Su2-lf8TbowpHZcS3XIRfQ,"[1, 4, 3, 4]",[Long unneCessary waits sometimes!\n ..... Poo...,"[{'useful': 2, 'funny': 1, 'cool': 0}, {'usefu...",domino's pizza,3100 e lake mead blvd
1824 e lake mead boulevard,xoYY0F5hrs00h285tWmZZA,"[4, 4, 4, 4, 3, 4, 4, 3, 4, 5, 2, 3, 3, 2, 4, ...","[New pizza place at the Wynn, named appropriat...","[{'useful': 2, 'funny': 1, 'cool': 1}, {'usefu...",carl's jr restaurants,1824 e lake mead boulevard


CODE FOR READING IN PICKLED OVERLAPPED FILES

In [32]:
try:
    with open('df_overlapped_inspections','rb') as f:
        df_overlapped_inspections = pickle.load(f)
        
except FileNotFoundError:
    print('can\'t find it')

In [33]:
try:
    with open('df_overlapped_reviews','rb') as f:
        df_overlapped_reviews = pickle.load(f)
        
except FileNotFoundError:
    print('can\'t find it')

In [34]:
df_lv_reviews = df_agg_revs.copy()
df_lv_reviews = df_lv_reviews[['address', 'name']]
df_lv_reviews.head()

,address,name
address,,
1916 village center cir,1916 village center cir,chianti cafe
2737 las vegas blvd n,2737 las vegas blvd n,rigos taco
2312 e craig rd,2312 e craig rd,mulligans bar & grill
3100 e lake mead blvd,3100 e lake mead blvd,domino's pizza
1824 e lake mead boulevard,1824 e lake mead boulevard,carl's jr restaurants


In [35]:
df_lv_reviews.shape

(4910, 2)

In [36]:
df_reviews_search = df_lv_reviews.set_index('address')
df_reviews_search.head()

,name
address,
1916 village center cir,chianti cafe
2737 las vegas blvd n,rigos taco
2312 e craig rd,mulligans bar & grill
3100 e lake mead blvd,domino's pizza
1824 e lake mead boulevard,carl's jr restaurants


In [37]:
df_reviews_search.shape

(4910, 1)

In [38]:
df_lv_violations = df_agg.copy()
df_lv_violations = df_lv_violations[['Restaurant Name', 'Address']]
df_lv_violations['Restaurant Name'] = df_lv_violations['Restaurant Name'].str.lower()
df_lv_violations['Address'] = df_lv_violations['Address'].str.lower()
df_lv_violations.head()

,Restaurant Name,Address
0,caesars seasonal venus pool bar deleted,3570 s las vegas blvd
1,mirage 1 oak small bar 2 original deleted,3400 s las vegas blvd
2,mirage 1 oak multilevel bar 4 deleted,3400 s las vegas blvd
3,golden gate beverage cart,1 fremont st
4,aramark nacho cart,850 n las vegas blvd


In [39]:
df_lv_violations.shape

(12926, 2)

In [40]:
df_violations_search = df_lv_violations.set_index('Address')
df_violations_search.head()
# len(df_lv_violations['Address'].unique())

,Restaurant Name
Address,
3570 s las vegas blvd,caesars seasonal venus pool bar deleted
3400 s las vegas blvd,mirage 1 oak small bar 2 original deleted
3400 s las vegas blvd,mirage 1 oak multilevel bar 4 deleted
1 fremont st,golden gate beverage cart
850 n las vegas blvd,aramark nacho cart


In [41]:
df_violations_search.shape

(12926, 1)

In [42]:
# bad_add = []
# for add_list in df_lv_violations['Address'].tolist():
#     if (len(set(add_list)) > 1):
#         bad_add.append(add_list)      

In [43]:
# def mergeDFs():
df_merged = pd.merge(df_reviews_search, df_violations_search, right_index=True, left_index=True)
# for index, row in df_reviews_search.iterrows():
#     if index in df_violations_search.index:
#         match_rows = df_violations_search.loc[:index]
df_merged.columns = ['yelp_name', 'hi_name']
df_merged.head()
        

,yelp_name,hi_name
1 s main st,subway,plaza hotel stuffed
1 s main st,subway,plaza steakhouse
1 s main st,subway,plaza steakhouse - bar
1 s main st,subway,plaza hotel veg prep / pantry
1 s main st,subway,plaza hotel derby lounge


In [44]:
df_merged.shape

(13773, 2)

In [45]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [46]:
df_merged['sim_score']= df_merged.apply(lambda row: similar(a = row['yelp_name'], b = row['hi_name']), axis = 1)

In [47]:
df_reviews_search.loc['1 s main st']

,name
address,
1 s main st,subway
1 s main st,center stage restaurant
1 s main st,mcdonald's
1 s main st,pop up pizza
1 s main st,zabas mexican grill
1 s main st,beer garden


In [48]:
df_reviews_search.loc['1 s main st'].shape

(6, 1)

In [49]:
df_violations_search.loc['1 s main st']

,Restaurant Name
Address,
1 s main st,plaza hotel stuffed
1 s main st,plaza steakhouse
1 s main st,plaza steakhouse - bar
1 s main st,plaza hotel veg prep / pantry
1 s main st,plaza hotel derby lounge
1 s main st,plaza hotel in room service
1 s main st,hash house a go go - main kitchen/storage
1 s main st,plaza hotel main kitchen
1 s main st,island sushi & hawaiian grill bar


In [50]:
df_violations_search.loc['1 s main st'].shape

(45, 1)

In [51]:
df_merged.loc['1 s main st']

,yelp_name,hi_name,sim_score
1 s main st,subway,plaza hotel stuffed,0.160000
1 s main st,subway,plaza steakhouse,0.181818
1 s main st,subway,plaza steakhouse - bar,0.285714
1 s main st,subway,plaza hotel veg prep / pantry,0.114286
1 s main st,subway,plaza hotel derby lounge,0.066667
1 s main st,subway,plaza hotel in room service,0.060606
1 s main st,subway,hash house a go go - main kitchen/storage,0.127660
1 s main st,subway,plaza hotel main kitchen,0.066667
1 s main st,subway,island sushi & hawaiian grill bar,0.205128
1 s main st,subway,gigi's,0.166667


In [52]:
df_merged_sorted = df_merged.sort(columns = 'sim_score', axis = 0, ascending=False)
df_merged_sorted.head(n=200)

/Users/takumihouse/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,yelp_name,hi_name,sim_score
2040 n nellis blvd,arandas taqueria,arandas taqueria,1.0
3981 e sunset rd,capriotti's sandwich shop,capriotti's sandwich shop,1.0
2893 w lake mead blvd,mega tom's burgers,mega tom's burgers,1.0
2885 e charleston blvd,pizza patron,pizza patron,1.0
3945 s durango dr,naked fish's sushi & grill,naked fish's sushi & grill,1.0
3220 n jones blvd,soul foo young,soul foo young,1.0
3950 n tenaya way,manhattan pizza iv,manhattan pizza iv,1.0
3950 schiff dr,hunan jiu jia,hunan jiu jia,1.0
2850 e tropicana ave,casa di amore,casa di amore,1.0
2840 e tropicana ave,bentley's coffee,bentley's coffee,1.0


In [53]:
df_sim_threshold = df_merged_sorted[df_merged_sorted['sim_score'] > 0.69]
# df[df.a > 1]
df_sim_threshold.sort(columns = 'sim_score', axis = 0, ascending=True).head(n=20)

/Users/takumihouse/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,yelp_name,hi_name,sim_score
313 n nellis blvd,hibachi grill & supreme buffet,hibachi grill - sushi bar,0.690909
5045 w tropicana ave,qdoba mexican grill,qdoba mexican grill tropicana / deca,0.690909
4945 w flamingo rd,mcdonald's,mcdonalds #17022,0.692308
7530 w washington ave,mcdonald's,mcdonalds #25295,0.692308
4855 s maryland pkwy,mcdonald's,mcdonalds #21201,0.692308
2233 n rampart blvd,pizza hut,pizza hut #026610,0.692308
7000 w charleston blvd,pizza hut,pizza hut #026588,0.692308
5111 boulder hwy,mcdonald's,mcdonalds #23749,0.692308
8090 blue diamond rd,pizza hut,pizza hut #026596,0.692308
255 e flamingo rd,tuscany gardens,tuscany edr,0.692308


In [54]:
# this is important, but need the indext as address for the merge later
# df_sim_threshold.reset_index(inplace = True)
# df_sim_threshold.columns = ['address', 'yelp_name', 'hi_name', 'sim_score']
# df_sim_threshold.head()

In [55]:
df_sim_threshold.shape

(1233, 3)

In [56]:
# df_lv_violations['Address'].map(len).max()

In [57]:
df_agg.head()

,Permit Number,Restaurant Name,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,4,A,05/17/2010 12:00:00 AM,4,A,NaN,Compliant,114,36.116156,-115.175058
1,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,10,A,07/30/2010 12:00:00 AM,10,A,NaN,Compliant,64114,36.120601,-115.176838
2,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,3,A,07/30/2010 12:00:00 AM,3,A,NaN,Compliant,13,36.120601,-115.176838
3,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,0,A,03/16/2010 12:00:00 AM,0,A,NaN,Compliant,NaN,36.171301,-115.146231
4,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,0,A,05/04/2011 12:00:00 AM,0,A,NaN,Compliant,NaN,36.180044,-115.131106


In [58]:
# def overlap_builder(address, row):
#     if 

In [59]:
df_agg['Address'] = df_agg['Address'].str.lower()
df_agg['Restaurant Name'] = df_agg['Restaurant Name'].str.lower()
print(df_agg.shape)
df_agg.set_index('Address', inplace = True)
print(df_agg.shape)
# df_agg_revs['address_lower'] = df_agg['Address'].str.lower()
# df_agg.head()

(12926, 15)
(12926, 14)


In [60]:
df_sim_threshold.head()

,yelp_name,hi_name,sim_score
2040 n nellis blvd,arandas taqueria,arandas taqueria,1.0
3981 e sunset rd,capriotti's sandwich shop,capriotti's sandwich shop,1.0
2893 w lake mead blvd,mega tom's burgers,mega tom's burgers,1.0
2885 e charleston blvd,pizza patron,pizza patron,1.0
3945 s durango dr,naked fish's sushi & grill,naked fish's sushi & grill,1.0


In [61]:
df_agg.head()

,Permit Number,Restaurant Name,Location Name,Category Name,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
Address,,,,,,,,,,,,,,
3570 s las vegas blvd,PR0000492,caesars seasonal venus pool bar deleted,CAESARS PALACE HOTEL & CASINO,Portable Unit,4,A,05/17/2010 12:00:00 AM,4,A,NaN,Compliant,114,36.116156,-115.175058
3400 s las vegas blvd,PR0000728,mirage 1 oak small bar 2 original deleted,MIRAGE HOTEL & CASINO,Bar / Tavern,10,A,07/30/2010 12:00:00 AM,10,A,NaN,Compliant,64114,36.120601,-115.176838
3400 s las vegas blvd,PR0000730,mirage 1 oak multilevel bar 4 deleted,MIRAGE HOTEL & CASINO,Bar / Tavern,3,A,07/30/2010 12:00:00 AM,3,A,NaN,Compliant,13,36.120601,-115.176838
1 fremont st,PR0000631,golden gate beverage cart,Golden Gate Hotel & Casino,Portable Unit,0,A,03/16/2010 12:00:00 AM,0,A,NaN,Compliant,NaN,36.171301,-115.146231
850 n las vegas blvd,PR0001171,aramark nacho cart,ARAMARK @ CASHMAN CENTER,Portable Unit,0,A,05/04/2011 12:00:00 AM,0,A,NaN,Compliant,NaN,36.180044,-115.131106


In [62]:
# df_agg_revs.reset_index(inplace = True)
# df_agg_revs.columns = ['name', 'business_id', 'stars', 'text', 'votes', 'address']
df_agg_revs.set_index('address', inplace = True)
df_agg_revs.head()

,business_id,stars,text,votes,name
address,,,,,
1916 village center cir,F8ezW6hYgmQYBmXOVjhytQ,"[3, 4, 2, 4, 3, 4, 1, 1, 1, 3, 4, 5, 1, 3, 4]",[This Jack's is not on the glitzy Las Vegas St...,"[{'useful': 4, 'funny': 3, 'cool': 4}, {'usefu...",chianti cafe
2737 las vegas blvd n,atLmZg76uDg0oKhKjHLMyg,"[2, 5, 1, 1, 5, 4, 3, 1, 1, 2, 1, 5, 3, 2, 2, ...",[Open 24 hours a day. Carna asada burrito is p...,"[{'useful': 1, 'funny': 0, 'cool': 0}, {'usefu...",rigos taco
2312 e craig rd,y7mody_dVe_2K8-wnnPtUw,"[5, 4, 4, 4, 5, 5, 5, 2, 4, 5, 5, 4, 5, 4, 4, ...","[The first time I was here, I ordered the Hawa...","[{'useful': 2, 'funny': 0, 'cool': 2}, {'usefu...",mulligans bar & grill
3100 e lake mead blvd,Su2-lf8TbowpHZcS3XIRfQ,"[1, 4, 3, 4]",[Long unneCessary waits sometimes!\n ..... Poo...,"[{'useful': 2, 'funny': 1, 'cool': 0}, {'usefu...",domino's pizza
1824 e lake mead boulevard,xoYY0F5hrs00h285tWmZZA,"[4, 4, 4, 4, 3, 4, 4, 3, 4, 5, 2, 3, 3, 2, 4, ...","[New pizza place at the Wynn, named appropriat...","[{'useful': 2, 'funny': 1, 'cool': 1}, {'usefu...",carl's jr restaurants


In [63]:
# print(df_agg_revs.shape)
# # df_agg_revs.set_index('name', inplace = True)
# print(df_agg_revs.shape)

In [64]:
df_inspections_merged = pd.merge(df_agg, df_sim_threshold, right_index=True, left_index=True)
# for index, row in df_reviews_search.iterrows():
#     if index in df_violations_search.index:
#         match_rows = df_violations_search.loc[:index]
df_inspections_merged = df_inspections_merged[df_inspections_merged['Restaurant Name'] == df_inspections_merged['hi_name']]
# df_inspections_merged.head()
df_inspections_merged.head()

,Permit Number,Restaurant Name,Location Name,Category Name,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y,yelp_name,hi_name,sim_score
1 s main st,PR0097511,zaba's mexican grill,ZABA'S MEXICAN GRILL,Restaurant,0,A,02/11/2016 12:00:00 AM,0,A,NaN,Compliant,2930,36.171678,-115.146414,zabas mexican grill,zaba's mexican grill,0.974359
1 s main st,PR0099965,pop up pizza,POP UP PIZZA @ PLAZA HOTEL,Restaurant,0,A,08/20/2016 12:00:00 AM,0,A,NaN,'A' Grade,NaN,36.171658,-115.146362,pop up pizza,pop up pizza,1.000000
100 n city pkwy,PR0011086,jason's deli #5,Jasons Deli Molasky Center,Restaurant,9,A,08/08/2016 12:00:00 AM,9,A,NaN,'A' Grade,2122909292829302955,36.174294,-115.147705,jason's deli,jason's deli #5,0.888889
1000 e charleston blvd,PR0007241,esmeraldas,ESMERALDAS,Restaurant,5,A,06/07/2016 12:00:00 AM,5,A,NaN,'A' Grade,209,36.158279,-115.140667,esmeralda's cafe,esmeraldas,0.769231
1000 n main st,PR0009534,la piazza bar,America's Best Value Inn,Bar / Tavern,3,A,11/04/2015 12:00:00 AM,3,A,NaN,Compliant,2911292829292956,36.181417,-115.139396,la piazza,la piazza bar,0.818182


In [65]:
df_overlapped_inspections = df_inspections_merged.drop(['yelp_name', 'hi_name', 'sim_score'], axis=1)
df_overlapped_inspections.reset_index(inplace = True)
df_overlapped_inspections.head()
df_overlapped_inspections.columns = ['address', 'Permit Number', 'Restaurant Name',
                                     'Location Name', 'Category Name', 'Current Demerits',
                                     'Current Grade', 'Date Current', 'Inspection Demerits',
                                     'Inspection Grade', 'Permit Status', 'Inspection Result',
                                     'Violations', 'X', 'Y']

In [66]:
df_overlapped_inspections.head()

,address,Permit Number,Restaurant Name,Location Name,Category Name,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,1 s main st,PR0097511,zaba's mexican grill,ZABA'S MEXICAN GRILL,Restaurant,0,A,02/11/2016 12:00:00 AM,0,A,NaN,Compliant,2930,36.171678,-115.146414
1,1 s main st,PR0099965,pop up pizza,POP UP PIZZA @ PLAZA HOTEL,Restaurant,0,A,08/20/2016 12:00:00 AM,0,A,NaN,'A' Grade,NaN,36.171658,-115.146362
2,100 n city pkwy,PR0011086,jason's deli #5,Jasons Deli Molasky Center,Restaurant,9,A,08/08/2016 12:00:00 AM,9,A,NaN,'A' Grade,2122909292829302955,36.174294,-115.147705
3,1000 e charleston blvd,PR0007241,esmeraldas,ESMERALDAS,Restaurant,5,A,06/07/2016 12:00:00 AM,5,A,NaN,'A' Grade,209,36.158279,-115.140667
4,1000 n main st,PR0009534,la piazza bar,America's Best Value Inn,Bar / Tavern,3,A,11/04/2015 12:00:00 AM,3,A,NaN,Compliant,2911292829292956,36.181417,-115.139396


In [67]:
df_reviews_merged = pd.merge(df_agg_revs, df_sim_threshold, right_index=True, left_index=True)
# for index, row in df_reviews_search.iterrows():
#     if index in df_violations_search.index:
#         match_rows = df_violations_search.loc[:index]
df_reviews_merged = df_reviews_merged[df_reviews_merged['name'] == df_reviews_merged['yelp_name']]
# df_inspections_merged.head()
df_reviews_merged.head()

,business_id,stars,text,votes,name,yelp_name,hi_name,sim_score
1 s main st,y18_x-NKD-V3EiLwkSj8Gw,"[3, 4, 3, 4, 1, 4, 4, 4, 4, 4, 3, 4, 4, 3, 1, ...",[This is one of the most popular places at eve...,"[{'useful': 4, 'funny': 1, 'cool': 3}, {'usefu...",pop up pizza,pop up pizza,pop up pizza,1.000000
1 s main st,LsArM1PhbqvXK3lDK-Ch_w,"[5, 5, 4, 4, 4, 5, 4, 5, 4, 3, 3, 5, 3, 3, 5, ...",[the best place for lunch around unlv\r\n\r\ng...,"[{'useful': 3, 'funny': 1, 'cool': 1}, {'usefu...",zabas mexican grill,zabas mexican grill,zaba's mexican grill,0.974359
100 n city pkwy,t2QVqqi4Mjf4y1uigc9Npg,"[4, 4, 5, 5, 4, 3, 4, 4, 4, 2, 4, 4, 4, 3, 5, ...",[I heard this place originated in Beverly Hill...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...",jason's deli,jason's deli,jason's deli #5,0.888889
1000 e charleston blvd,vh9CQdZj-1N21kW0qgKmnQ,"[3, 3, 4, 4, 4, 4, 5, 3, 4, 3, 4, 2, 5, 5, 4, ...",[Chopstix is a New York style Chinese restaura...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...",esmeralda's cafe,esmeralda's cafe,esmeraldas,0.769231
1000 n main st,uCx8VZw2oiAdi7wFnoPs-w,"[4, 3, 3, 1]","[Clean, sells 20 pounds of ice for only $2 for...","[{'useful': 0, 'funny': 1, 'cool': 1}, {'usefu...",la piazza,la piazza,la piazza bar,0.818182


In [68]:
df_overlapped_reviews = df_reviews_merged.drop(['yelp_name', 'hi_name', 'sim_score'], axis=1)
df_overlapped_reviews.reset_index(inplace = True)
df_overlapped_reviews.columns = ['address', 'business_id', 'stars',
                                     'text', 'votes', '	name']
df_overlapped_reviews.head()

,address,business_id,stars,text,votes,name
0,1 s main st,y18_x-NKD-V3EiLwkSj8Gw,"[3, 4, 3, 4, 1, 4, 4, 4, 4, 4, 3, 4, 4, 3, 1, ...",[This is one of the most popular places at eve...,"[{'useful': 4, 'funny': 1, 'cool': 3}, {'usefu...",pop up pizza
1,1 s main st,LsArM1PhbqvXK3lDK-Ch_w,"[5, 5, 4, 4, 4, 5, 4, 5, 4, 3, 3, 5, 3, 3, 5, ...",[the best place for lunch around unlv\r\n\r\ng...,"[{'useful': 3, 'funny': 1, 'cool': 1}, {'usefu...",zabas mexican grill
2,100 n city pkwy,t2QVqqi4Mjf4y1uigc9Npg,"[4, 4, 5, 5, 4, 3, 4, 4, 4, 2, 4, 4, 4, 3, 5, ...",[I heard this place originated in Beverly Hill...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...",jason's deli
3,1000 e charleston blvd,vh9CQdZj-1N21kW0qgKmnQ,"[3, 3, 4, 4, 4, 4, 5, 3, 4, 3, 4, 2, 5, 5, 4, ...",[Chopstix is a New York style Chinese restaura...,"[{'useful': 0, 'funny': 0, 'cool': 0}, {'usefu...",esmeralda's cafe
4,1000 n main st,uCx8VZw2oiAdi7wFnoPs-w,"[4, 3, 3, 1]","[Clean, sells 20 pounds of ice for only $2 for...","[{'useful': 0, 'funny': 1, 'cool': 1}, {'usefu...",la piazza


In [69]:
with open('df_overlapped_reviews','wb') as f:
    pickle.dump(df_overlapped_reviews, f)
with open('df_overlapped_inspections','wb') as f:
    pickle.dump(df_overlapped_inspections, f)